In [ ]:
import tifffile as tiff
with tiff.TiffFile(r"train\Germany_Training_Public\PRE-event\10500500C4DD7000_0_17_66.tif") as tif:
    # plt.figure(figsize=(10, 10))
    # plt.imshow(image[:, :, 0: 4])
    # plt.show()
    for key in tif.pages[0].tags.keys():
        print('key: ', key, 'value: {0}'.format(tif.pages[0].tags[key].value))
        # print(tif.pages[0].tags[key].value)
        
    print(tif.pages[0].tags[33922].value)
        

key:  256 value: 1300
key:  257 value: 1300
key:  258 value: (8, 8, 8)
key:  259 value: 5
key:  262 value: 2
key:  277 value: 3
key:  284 value: 2
key:  317 value: 1
key:  322 value: 256
key:  323 value: 256
key:  324 value: (1278, 47510, 87126, 120140, 160500, 200859, 205429, 253547, 291200, 324329, 363501, 403573, 408372, 459193, 512828, 556362, 608381, 653820, 657393, 697148, 754534, 804219, 850520, 900108, 905415, 958430, 1017196, 1067598, 1111401, 1160943, 1166394, 1170864, 1175341, 1178792, 1182084, 1186453, 1187304, 1236400, 1278877, 1316632, 1358665, 1401880, 1406608, 1455614, 1497987, 1536803, 1578396, 1619530, 1624405, 1676076, 1730557, 1777386, 1829614, 1875566, 1879248, 1921980, 1979276, 2027257, 2071695, 2120308, 2125556, 2178240, 2235981, 2285692, 2328659, 2376208, 2381137, 2385494, 2390172, 2393733, 2397240, 2401605, 2402451, 2445932, 2483382, 2518822, 2557908, 2594998, 2599429, 2647529, 2685029, 2715695, 2752937, 2792343, 2797087, 2847913, 2900357, 2942546, 2992774, 303

In [ ]:
import os
import rasterio as rio
import matplotlib.pyplot as plt
from PIL import Image
import tifffile as tiff
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, LineString

def convert(tiff_folder, annot_folder, X_folder, Y_folder, X_folder_val, Y_folder_val, prefix):
    os.makedirs(X_folder, exist_ok=True)
    os.makedirs(Y_folder, exist_ok=True)
    os.makedirs(X_folder_val, exist_ok=True)
    os.makedirs(Y_folder_val, exist_ok=True)
    
    tiffs = os.listdir(tiff_folder)
    annots = os.listdir(annot_folder)
    i = 0
    for tiff_file, annot_file in zip(tiffs, annots):
        if tiff_file.endswith(".tif") or tiff_file.endswith("x.tiff"):
            # Open the TIFF file
            tiff_path = os.path.join(tiff_folder, tiff_file)
            with rio.open(tiff_path) as src:
                image = src.read().transpose((1, 2, 0))
            image = Image.fromarray(image).resize((512, 512))

            # Create the output filename
            base_name = os.path.splitext(tiff_file)[0]
            base_name = base_name.split('/')[-1]
            base_name = base_name.split('_')[1:]
            base_name = '_'.join(base_name)
            output_filename = f"{prefix}_{base_name}.png"
            if i % 8 == 0:
                output_path = os.path.join(X_folder_val, output_filename)
            else:
                output_path = os.path.join(X_folder, output_filename)

            # Save the image as PNG
            image.save(output_path)
            
            resolution = 0
            scale = 0
            initial_x = 0
            initial_y = 0
            with tiff.TiffFile(tiff_path) as tif:
                resolution = tif.pages[0].tags[256].value
                scale = tif.pages[0].tags[33550].value[0]
                initial_x = tif.pages[0].tags[33922].value[3]
                initial_y = tif.pages[0].tags[33922].value[4]
        
        if annot_file.endswith(".geojson"):
            try:
                # Read the GeoJSON file
                geojson_path = os.path.join(annot_folder, annot_file)
                lab = gpd.read_file(geojson_path)
                lab.set_crs(epsg=4326, inplace=True)

                # Filter buildings and roads
                buildings = lab[lab['building'] == 'yes']
                primary_roads = lab[(lab['highway'].notna()) & (lab['building'].isna()) & (lab["highway"] == "primary")]
                secondary_roads = lab[(lab['highway'].notna()) & (lab['building'].isna()) & (lab["highway"] == "secondary")]
                tertiary_roads = lab[(lab['highway'].notna()) & (lab['building'].isna()) & (lab["highway"] == "tertiary")]
                residential_roads = lab[(lab['highway'].notna()) & (lab['building'].isna()) & (lab["highway"] == "residential")]
                other_roads = lab[(lab['highway'].notna()) & (lab['building'].isna()) & (lab["highway"] != "primary") & (lab["highway"] != "secondary") & (lab["highway"] != "tertiary") & (lab["highway"] != "residential")]

                # Create a plot
                # figsize=(13.33, 13.33)
                img_1024_size = 13.33
                img_size = 6.66
                fig, ax = plt.subplots(figsize=(img_size, img_size))
                edge_x = [initial_x, initial_x + resolution*scale, initial_x + resolution*scale, initial_x]
                edge_y = [initial_y - resolution*scale, initial_y - resolution*scale, initial_y, initial_y]
                # print(edge_x, edge_y)
                # ax.set_ylim(initial_y, initial_y + resolution*scale)
                # ax.set_xlim(initial_x, initial_x + resolution*scale)

                ax.cla()
                fig.set_facecolor((0, 1, 0, 1))
                # ax = lab[lab['building']=='yes'].plot(color = 'b',label = 'buildings',figsize = (12,12))
                # lab[lab['building']!='yes'].geometry.buffer(0.00001).plot(color = 'r',ax = ax)
                # Plot buildings in 
                
                if not buildings.empty:
                    buildings.plot(ax=ax, color= (0, 0, 1, 1), label='buildings')

                # Plot roads in black
                if not primary_roads.empty:
                    primary_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth=15 * (img_size / img_1024_size), label='roads')
                    
                if not secondary_roads.empty:
                    secondary_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth=11 * (img_size / img_1024_size), label='roads')
                    
                if not secondary_roads.empty:
                    secondary_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth= 9 * (img_size / img_1024_size), label='roads')
                    
                if not tertiary_roads.empty:
                    tertiary_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth= 8 * (img_size / img_1024_size), label='roads')
                    
                if not residential_roads.empty:
                    residential_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth= 6 * (img_size / img_1024_size), label='roads')
                    
                if not other_roads.empty:
                    other_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth=3 * (img_size / img_1024_size), label='road')
                ax.scatter(edge_x, edge_y, color = (0, 1, 0, 1), marker = '.')
                
                ax.set_axis_off()
                ax.set_aspect('equal')
                # print(fig.get_figwidth())

                # Create the output filename
                base_name = os.path.splitext(annot_file)[0]
                output_filename = f"{prefix}_{base_name}.png"
                if i % 8 == 0:
                    output_path = os.path.join(Y_folder_val, output_filename)
                else:
                    output_path = os.path.join(Y_folder, output_filename)
                # print(ax.get_xscale(), ax.get_aspect(), ax.get_xlim(), ax.get_ylim())
                # Save the image
                # plt.gca().set_axis_off()

                plt.margins(0,0)
                plt.savefig(output_path, pad_inches = 0, bbox_inches='tight', dpi = 100)
                plt.plot()
                plt.close()
                
                i += 1

            except Exception as e:
                print(f"Error processing {annot_file}: {e}")
    
    print("Conversion completed.")
    
    

In [ ]:
tiff_folder = 'train/Germany_Training_Public/PRE-event'
X_folder = 'X_images'
X_folder_val = 'X_val'
annot_folder = 'train/Germany_Training_Public/annotations'
Y_folder = 'Y_train'
Y_folder_val = 'Y_val'
prefix = 'Germany'

convert(tiff_folder, annot_folder, X_folder, Y_folder, X_folder_val, Y_folder_val, prefix)

In [ ]:
import os
import rasterio as rio
import matplotlib.pyplot as plt
from PIL import Image
import tifffile as tiff
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, LineString

def convert(tiff_folder, annot_folder, X_folder, Y_folder, X_folder_val, Y_folder_val, prefix):
    os.makedirs(X_folder, exist_ok=True)
    os.makedirs(Y_folder, exist_ok=True)
    os.makedirs(X_folder_val, exist_ok=True)
    os.makedirs(Y_folder_val, exist_ok=True)
    
    tiffs = os.listdir(tiff_folder)
    tiffs = sorted(tiffs, key = lambda x: '_'.join(os.path.splitext(x)[0].split('_')[1:]))
    print(tiffs[10:20])
    annots = sorted(os.listdir(annot_folder))
    print(annots[10:20])
    i = 0
    for tiff_file, annot_file in zip(tiffs, annots):
        if tiff_file.endswith(".tif") or tiff_file.endswith("x.tiff"):
            # Open the TIFF file
            tiff_path = os.path.join(tiff_folder, tiff_file)
            with rio.open(tiff_path) as src:
                image = src.read().transpose((1, 2, 0))
            image = Image.fromarray(image).resize((512, 512))

            # Create the output filename
            base_name = os.path.splitext(tiff_file)[0]
            base_name = base_name.split('/')[-1]
            base_name = base_name.split('_')[1:]
            base_name = '_'.join(base_name)
            output_filename = f"{prefix}_{base_name}.png"
            if i % 8 == 0:
                output_path = os.path.join(X_folder_val, output_filename)
            else:
                output_path = os.path.join(X_folder, output_filename)

            # Save the image as PNG
            image.save(output_path)
            
            resolution = 0
            scale = 0
            initial_x = 0
            initial_y = 0
            with tiff.TiffFile(tiff_path) as tif:
                resolution = tif.pages[0].tags[256].value
                scale = tif.pages[0].tags[33550].value[0]
                initial_x = tif.pages[0].tags[33922].value[3]
                initial_y = tif.pages[0].tags[33922].value[4]
        
        if annot_file.endswith(".geojson"):
            try:
                # Read the GeoJSON file
                geojson_path = os.path.join(annot_folder, annot_file)
                lab = gpd.read_file(geojson_path)
                lab.set_crs(epsg=4326, inplace=True)

                # Filter buildings and roads
                buildings = lab[lab['building'] == 'yes']
                primary_roads = lab[(lab['highway'].notna()) & (lab['building'].isna()) & (lab["highway"] == "primary")]
                secondary_roads = lab[(lab['highway'].notna()) & (lab['building'].isna()) & (lab["highway"] == "secondary")]
                tertiary_roads = lab[(lab['highway'].notna()) & (lab['building'].isna()) & (lab["highway"] == "tertiary")]
                residential_roads = lab[(lab['highway'].notna()) & (lab['building'].isna()) & (lab["highway"] == "residential")]
                other_roads = lab[(lab['highway'].notna()) & (lab['building'].isna()) & (lab["highway"] != "primary") & (lab["highway"] != "secondary") & (lab["highway"] != "tertiary") & (lab["highway"] != "residential")]

                # Create a plot
                # figsize=(13.33, 13.33)
                img_1024_size = 13.33
                img_size = 6.66
                fig, ax = plt.subplots(figsize=(img_size, img_size))
                edge_x = [initial_x, initial_x + resolution*scale, initial_x + resolution*scale, initial_x]
                edge_y = [initial_y - resolution*scale, initial_y - resolution*scale, initial_y, initial_y]
                # print(edge_x, edge_y)
                # ax.set_ylim(initial_y, initial_y + resolution*scale)
                # ax.set_xlim(initial_x, initial_x + resolution*scale)

                ax.cla()
                fig.set_facecolor((0, 1, 0, 1))
                # ax = lab[lab['building']=='yes'].plot(color = 'b',label = 'buildings',figsize = (12,12))
                # lab[lab['building']!='yes'].geometry.buffer(0.00001).plot(color = 'r',ax = ax)
                # Plot buildings in 
                
                if not buildings.empty:
                    buildings.plot(ax=ax, color= (0, 0, 1, 1), label='buildings')

                # Plot roads in black
                if not primary_roads.empty:
                    primary_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth=15 * (img_size / img_1024_size), label='roads')
                    
                if not secondary_roads.empty:
                    secondary_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth=11 * (img_size / img_1024_size), label='roads')
                    
                if not secondary_roads.empty:
                    secondary_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth= 9 * (img_size / img_1024_size), label='roads')
                    
                if not tertiary_roads.empty:
                    tertiary_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth= 8 * (img_size / img_1024_size), label='roads')
                    
                if not residential_roads.empty:
                    residential_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth= 6 * (img_size / img_1024_size), label='roads')
                    
                if not other_roads.empty:
                    other_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth=3 * (img_size / img_1024_size), label='road')
                ax.scatter(edge_x, edge_y, color = (0, 1, 0, 1), marker = '.')
                
                ax.set_axis_off()
                ax.set_aspect('equal')
                # print(fig.get_figwidth())

                # Create the output filename
                base_name = os.path.splitext(annot_file)[0]
                output_filename = f"{prefix}_{base_name}.png"
                if i % 8 == 0:
                    output_path = os.path.join(Y_folder_val, output_filename)
                else:
                    output_path = os.path.join(Y_folder, output_filename)
                # print(ax.get_xscale(), ax.get_aspect(), ax.get_xlim(), ax.get_ylim())
                # Save the image
                # plt.gca().set_axis_off()

                plt.margins(0,0)
                plt.savefig(output_path, pad_inches = 0, bbox_inches='tight', dpi = 100)
                plt.plot()
                plt.close()
                
                i += 1

            except Exception as e:
                print(f"Error processing {annot_file}: {e}")
    
    print("Conversion completed.")
    
    

In [ ]:
tiff_folder = 'train/Louisiana-East_Training_Public/PRE-event'
X_folder = 'X_images'
X_folder_val = 'X_val'
annot_folder = 'train/Louisiana-East_Training_Public/annotations'
Y_folder = 'Y_train'
Y_folder_val = 'Y_val'
prefix = 'Louisiana'

convert(tiff_folder, annot_folder, X_folder, Y_folder, X_folder_val, Y_folder_val, prefix)

In [ ]:
import os
import numpy as np
from PIL import Image


def process_png_masks(image_folder, output_folder):
    """
    Opens all PNG images in a folder, determines the dominant color channel
    (red, green, or blue), divides them into 4 patches, and saves corresponding
    single-channel NumPy arrays with values 0, 1, or 2, along with a demarcation patch,
    in a specified output folder.

    Args:
        image_folder (str): Path to the folder containing PNG images.
        output_folder (str): Path to the folder where processed arrays will be saved.
    """

    os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist

    for filename in os.listdir(image_folder):
        if filename.endswith(".png"):
            image_path = os.path.join(image_folder, filename)

            try:
                img = Image.open(image_path)
                img_array = np.array(img)

                # Determine dominant color channel efficiently
                dominant_channel = np.argmax(img_array[:, :, :3], axis=2)

                # Split image into 4 patches (assuming divisible by 256)
                if img_array.shape[0] % 256 == 0 and img_array.shape[1] % 256 == 0:
                    patch_size = 256
                    num_patches = 4

                    for i in range(num_patches):
                        x1, y1 = i // 2 * patch_size, i % 2 * patch_size
                        x2, y2 = x1 + patch_size, y1 + patch_size
                        patch = dominant_channel[x1:x2, y1:y2]

                        # Save patch with informative filename
                        output_filename = f"{os.path.splitext(filename)[0]}_patch_{i}.npy"
                        output_path = os.path.join(output_folder, output_filename)
                        np.savez_compressed(output_path, patch)
                else:
                    print(f"Image {filename} dimensions ({img_array.shape[0]},"
                          f" {img_array.shape[1]}) not divisible by 256 for patching.")

            except (OSError, ValueError) as e:
                print(f"Error processing image {filename}: {e}")




In [ ]:
image_folder = "Y_train"  # Replace with your image folder path
output_folder = "Y_train_np"  # Replace with your output folder path
process_png_masks(image_folder, output_folder)

In [ ]:
image_folder = "Y_val"  # Replace with your image folder path
output_folder = "Y_val_np"  # Replace with your output folder path
process_png_masks(image_folder, output_folder)

In [ ]:
import os
import numpy as np
from PIL import Image


def process_and_save_png_patches(image_folder, output_folder):
    """
    Opens all PNG images in a folder, resizes them to 512x512 if necessary,
    divides them into 4 patches of 256x256, creates reconstructions as RGB images,
    and saves them as separate PNG files.

    Args:
        image_folder (str): Path to the folder containing PNG images.
        output_folder (str): Path to the folder where processed PNGs will be saved.
    """

    os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist

    for filename in os.listdir(image_folder):
        if filename.endswith(".png"):
            image_path = os.path.join(image_folder, filename)

            try:
                img = Image.open(image_path)

                # Resize to 512x512 if needed (optional)
                if img.size != (512, 512):
                    img = img.resize((512, 512))

                img_array = np.array(img)


                # Split image into 4 patches (assuming divisible by 256)
                if img_array.shape[0] % 256 == 0 and img_array.shape[1] % 256 == 0:
                    patch_size = 256
                    num_patches = 4

                    for i in range(num_patches):
                        x1, y1 = i // 2 * patch_size, i % 2 * patch_size
                        x2, y2 = x1 + patch_size, y1 + patch_size
                        patch = img_array[x1:x2, y1:y2]

                        # Convert patch to RGB image and save as PNG
                        rgb_image = Image.fromarray(patch)
                        patch_output_filename = f"{os.path.splitext(filename)[0]}_patch_{i}.png"
                        patch_output_path = os.path.join(output_folder, patch_output_filename)
                        rgb_image.save(patch_output_path)
            except (OSError, ValueError) as e:
                print(f"Error processing image {filename}: {e}")


In [ ]:
image_folder = "X_images"
output_folder = "X_img_patches"
process_and_save_png_patches(image_folder, output_folder)

In [ ]:
image_folder = "X_val"
output_folder = "X_val_patches"
process_and_save_png_patches(image_folder, output_folder)

In [ ]:
data = np.load("Y_val_np\Germany_0_15_63.npy")
print(data[90:100, 122: 140])

[[0 1 1 1 1 1 1 1 2 1 1 2 2 2 2 2 2 2]
 [0 0 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2]
 [0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2]
 [1 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 1]
 [1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1]]
